In [38]:
import pandas as pd
import numpy as np
import requests
from datetime import date, datetime


In [39]:
url = 'https://en.shmet.com/api/rest/enweb/spot/getSpotPrice?code=baseMetal&size=10&currentLength=0'

In [40]:
responce = requests.get(url)

In [41]:
day_df = pd.DataFrame(responce.json()['data'])
day_df['date'] = date.today()
day_df

,scId,code,type,pos,name,low,high,middle,change,time,unit,showIcon,date
0,811,HomeEn,baseMetal,1,Cu cathode 1#,70150,70350,70250,+75,11:30,Yuan/MT,N,2023-09-04
1,794,HomeEn,baseMetal,2,"Al ingot A00, East China",19390,19430,19410,-200,10:15,Yuan/MT,N,2023-09-04
2,802,HomeEn,baseMetal,3,"Pb ingot 1#, east China",16350,16450,16400,-50,10:15,Yuan/MT,N,2023-09-04
3,795,HomeEn,baseMetal,5,"Zn ingot 0#, Shanghai",21490,21590,21540,+130,10:15,Yuan/MT,N,2023-09-04
4,808,HomeEn,baseMetal,6,"Zn ingot 1#, Shanghai",21420,21520,21470,+130,10:15,Yuan/MT,N,2023-09-04
5,810,HomeEn,baseMetal,7,"Zn ingot imported, Shanghai",21460,21560,21510,+130,10:15,Yuan/MT,N,2023-09-04
6,799,HomeEn,baseMetal,8,"Tin ingot 1#(99.9%),East China",216500,218000,217250,+250,10:15,Yuan/MT,N,2023-09-04
7,813,HomeEn,baseMetal,9,"Ni cathodes 1#, Shanghai",170800,173500,172150,+3400,10:15,Yuan/MT,N,2023-09-04
8,801,HomeEn,baseMetal,10,"Ni cathodes(Russian), Shanghai",170800,171000,170900,+3550,10:15,Yuan/MT,N,2023-09-04
9,803,HomeEn,baseMetal,11,"Ni cathodes(Jinchuan), Shanghai",173300,173500,173400,+3250,10:15,Yuan/MT,N,2023-09-04


In [42]:
cooper_row = day_df[day_df['name'].str.contains("cu", case=False)]
cooper_row

,scId,code,type,pos,name,low,high,middle,change,time,unit,showIcon,date
0,811,HomeEn,baseMetal,1,Cu cathode 1#,70150,70350,70250,+75,11:30,Yuan/MT,N,2023-09-04


In [47]:
result = cooper_row[['date', 'middle', 'unit']]
result = result.rename(columns={'middle':'price'})
result['date'] = pd.to_datetime(result['date'])
result

,date,price,unit
0,2023-09-04,70250,Yuan/MT


In [48]:
hist_data = pd.read_excel('./data/shmet_historical.xlsx', index_col=0)
hist_data

,date,price,unit
0,2023-09-01,70175,Yuan/MT
1,2023-08-31,70075,Yuan/MT
2,2023-08-30,69880,Yuan/MT
3,2023-08-29,69525,Yuan/MT
4,2023-08-28,69530,Yuan/MT
...,...,...,...
675,2020-01-17,48930,Yuan/MT
676,2020-01-16,48950,Yuan/MT
677,2020-01-15,49060,Yuan/MT
678,2020-01-14,48990,Yuan/MT


In [52]:
pd.concat([result, hist_data], axis=0).reset_index(drop=True)

,date,price,unit
0,2023-09-04,70250,Yuan/MT
1,2023-09-01,70175,Yuan/MT
2,2023-08-31,70075,Yuan/MT
3,2023-08-30,69880,Yuan/MT
4,2023-08-29,69525,Yuan/MT
...,...,...,...
676,2020-01-17,48930,Yuan/MT
677,2020-01-16,48950,Yuan/MT
678,2020-01-15,49060,Yuan/MT
679,2020-01-14,48990,Yuan/MT


In [57]:
def shmet_optimized():
    url = 'https://en.shmet.com/api/rest/enweb/spot/getSpotPrice?code=baseMetal&size=10&currentLength=0'
    responce = requests.get(url)

    day_df = pd.DataFrame(responce.json()['data'])
    day_df['date'] = date.today()
    cooper_row = day_df[day_df['name'].str.contains("cu", case=False)]

    result = cooper_row[['date', 'middle', 'unit']]
    result['date'] = pd.to_datetime(result['date'])
    result = result.rename(columns={'middle': 'price'})

    hist_data = pd.read_excel('./data/shmet_historical.xlsx', index_col=0)

    new_df = pd.concat([result, hist_data], axis=0).reset_index(drop=True)
    new_df.drop_duplicates(inplace=True)

    with pd.ExcelWriter(
        "../parser_beta/data/shmet_historical.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD") as writer:
        new_df.to_excel(writer, sheet_name='SHMET')
        
    final = pd.read_excel('./data/shmet_historical.xlsx', index_col=0)
    final.drop_duplicates(inplace=True)
    
    with pd.ExcelWriter(
        "../parser_beta/data/shmet_historical.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD") as writer:
        final.to_excel(writer, sheet_name='SHMET')
        
    google_upload(final, 'SHMET')

    return final

In [58]:
shmet_optimized()

/var/folders/hp/79qzs1rx74qgjz43pgfcb1340000gn/T/ipykernel_5675/2129858424.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['date'] = pd.to_datetime(result['date'])


,date,price,unit
0,2023-09-04,70250,Yuan/MT
3,2023-09-01,70175,Yuan/MT
4,2023-08-31,70075,Yuan/MT
5,2023-08-30,69880,Yuan/MT
6,2023-08-29,69525,Yuan/MT
...,...,...,...
678,2020-01-17,48930,Yuan/MT
679,2020-01-16,48950,Yuan/MT
680,2020-01-15,49060,Yuan/MT
681,2020-01-14,48990,Yuan/MT
